# Collapsar dynamics 

This notebook contains the calculations performed in the paper arXiv:2312.10400 

In [1]:
version()

'SageMath version 9.8, Release Date: 2023-02-11'

First we set up the notebook to display mathematical objects using LaTeX formatting:

In [2]:
%display latex

# ----- To make the notebook wider ------
from IPython.display import display, HTML
display(HTML("<style>.container { width:50% !important; }</style>"))

To speed up computations,run them in parallel on 8 threads:

In [3]:
# Parallelism().set(nproc=8)

## Spacetime

We declare the spacetime manifold $M$ with dim = 4 and Lorentzian signature:

In [4]:
M = Manifold(4, 'M', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


Declare the Boyer-Lindquist coordinates $(t,r,\theta,\phi)$ as a chart on $M$:

In [5]:
BL.<t,r,th,ph> = M.chart(r't r th:(0,pi):\theta ph:(0,2*pi):\phi')
BL

Chart (M, (t, r, th, ph))

The chart with its induced basis

In [6]:
BL.frame()

Coordinate frame (M, (∂/∂t,∂/∂r,∂/∂th,∂/∂ph))

As an example this is how to print the timelike Killing field defined in the paper

In [7]:
xi = BL.frame()[0]; xi   

Vector field ∂/∂t on the 4-dimensional Lorentzian manifold M

Print the chart range

In [8]:
BL.coord_range()

t: (-oo, +oo); r: (-oo, +oo); th: (0, pi); ph: (0, 2*pi)

## Define the Kerr metric

We define the metric $g$ using its components w.r.t. the Boyer-Lindquist coordinates:

In [9]:
g = M.lorentzian_metric(name = 'g', signature='positive')
m, a = var('m a')

# ------ Set a to zero for Schwarzschild metric ------
# a = 0
rho2 = r^2 + (a*cos(th))^2
Delta = r^2 - 2*m*r + a^2

g[0,0] = -(1 - 2*m*r/rho2)
g[0,3] = -2*a*m*r*sin(th)^2/rho2
g[1,1] = rho2/Delta
g[2,2] = rho2
g[3,3] = (r^2 + a^2 + 2*m*r*(a*sin(th))^2/rho2)*sin(th)^2

# # # -------- E.g. for Minkowski metric ------------
# g[0,3] = 0
# g[0,0] = -1
# g[1,1] = +1
# g[2,2] = +1
# g[3,3] = +1

In [10]:
g.signature()

2

In [11]:
g.display()

g = (2*m*r/(a^2*cos(th)^2 + r^2) - 1) dt⊗dt - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dt⊗dph + (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2) dr⊗dr + (a^2*cos(th)^2 + r^2) dth⊗dth - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dph⊗dt + (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2 dph⊗dph

 In matrix form

In [12]:
g[:]

[                                  2*m*r/(a^2*cos(th)^2 + r^2) - 1                                                                 0                                                                 0                          -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2)]
[                                                                0                         (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2)                                                                 0                                                                 0]
[                                                                0                                                                 0                                               a^2*cos(th)^2 + r^2                                                                 0]
[                         -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2)                                                                 0                                                                 0 (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2]

The inverse metric

In [13]:
g.inverse()[:]

[-(2*a^2*m*r*sin(th)^2 + a^2*r^2 + r^4 + (a^4 + a^2*r^2)*cos(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2)                                                                                                                                    0                                                                                                                                    0                                                           -2*a*m*r/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2)]
[                                                                                                                                   0                                                                                            (a^2 - 2*m*r + r^2)/(a^2*cos(th)^2 + r^2)                                                                                                                                    0                                                                                                                                    0]
[                                                                                                                                   0                                                                                                                                    0                                                                                                              1/(a^2*cos(th)^2 + r^2)                                                                                                                                    0]
[                                                          -2*a*m*r/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2)                                                                                                                                    0                                                                                                                                    0    (a^2*cos(th)^2 - 2*m*r + r^2)/(2*a^2*m*r*sin(th)^4 - (2*a^2*m*r - a^2*r^2 + 2*m*r^3 - r^4 - (a^4 + a^2*r^2)*cos(th)^2)*sin(th)^2)]

### Some practice exercises that can be skipped

Calculating inner product $g_{ab} \xi^a \xi^b$

In [14]:
g(xi, xi).display() 

g(∂/∂t,∂/∂t): M → ℝ
   (t, r, th, ph) ↦ 2*m*r/(a^2*cos(th)^2 + r^2) - 1

Another way to calculate inner product $\xi^a \xi_a$

In [15]:
xi.contract(xi.down(g)).display()  # Know how to lower/raise index would be helpful later

g(∂/∂t,∂/∂t): M → ℝ
   (t, r, th, ph) ↦ 2*m*r/(a^2*cos(th)^2 + r^2) - 1

Normalising $\xi$ so that the inner product is equal to -1

In [16]:
xi_unit = xi/sqrt(-g(xi,xi)); xi_unit.display()

sqrt(a^2*cos(th)^2 + r^2)/sqrt(a^2*cos(th)^2 - 2*m*r + r^2) ∂/∂t

Calculating its inner product w.r.t. metric $g$ (yet another way!)

In [17]:
xi_unit.dot(xi_unit, metric=g).display() 

M → ℝ
(t, r, th, ph) ↦ -1

or the same old way

In [18]:
g(xi_unit, xi_unit).display()

M → ℝ
(t, r, th, ph) ↦ -1

### End of practice exercises

Decalare some variable quantities

In [19]:
gamma, Omega = var('gamma Omega') 

Decalre a vector field on M. With appropriate boundary conditions ($t, r, \theta, \phi$), this will be the 4-velocity of the shell element defined in the paper (see sectoin 2.2.3)

In [20]:
u = M.vector_field('u')  
u[0] = gamma
u[1] = 0
u[2] = 0
u[3] = gamma*Omega   # Set to zero to ease calculation (This is the assumed condition in paper)

In [21]:
u.display()

u = gamma ∂/∂t + Omega*gamma ∂/∂ph

### Define a (2,0) tensor field on M to represent the energy-momentum tensor

In [22]:
T = M.tensor_field(2,0, name='T^{ab}') ; T

Tensor field T^{ab} of type (2,0) on the 4-dimensional Lorentzian manifold M

Fill data in $T^{ab}$, which we assume to be a dust equation of state

In [23]:
rho = M.scalar_field(function('rho')(r), name=r'\rho')
T = rho * u*u
T.display()

\rho*u⊗u = gamma^2*rho(r) ∂/∂t⊗∂/∂t + Omega*gamma^2*rho(r) ∂/∂t⊗∂/∂ph + Omega*gamma^2*rho(r) ∂/∂ph⊗∂/∂t + Omega^2*gamma^2*rho(r) ∂/∂ph⊗∂/∂ph

In [24]:
# T.display_comp()

Calculating $u_a$

In [25]:
u.down(g).display()

-(2*Omega*a*gamma*m*r*sin(th)^2 + a^2*gamma*cos(th)^2 - 2*gamma*m*r + gamma*r^2)/(a^2*cos(th)^2 + r^2) dt + (2*Omega*a^2*gamma*m*r*sin(th)^4 + (Omega*a^2*gamma*r^2 + Omega*gamma*r^4 - 2*a*gamma*m*r + (Omega*a^4*gamma + Omega*a^2*gamma*r^2)*cos(th)^2)*sin(th)^2)/(a^2*cos(th)^2 + r^2) dph

Calculating $T^{ab}u_a$ 

In [26]:
mass_energy_currrent = T.contract(u.down(g))
mass_energy_currrent.display()

(2*Omega^2*a^2*gamma^3*m*r*rho(r)*sin(th)^4 - a^2*gamma^3*cos(th)^2*rho(r) + ((Omega^2*a^4*gamma^3 + Omega^2*a^2*gamma^3*r^2)*cos(th)^2*rho(r) + (Omega^2*a^2*gamma^3*r^2 + Omega^2*gamma^3*r^4 - 4*Omega*a*gamma^3*m*r)*rho(r))*sin(th)^2 + (2*gamma^3*m*r - gamma^3*r^2)*rho(r))/(a^2*cos(th)^2 + r^2) ∂/∂t + (2*Omega^3*a^2*gamma^3*m*r*rho(r)*sin(th)^4 - Omega*a^2*gamma^3*cos(th)^2*rho(r) + ((Omega^3*a^4*gamma^3 + Omega^3*a^2*gamma^3*r^2)*cos(th)^2*rho(r) + (Omega^3*a^2*gamma^3*r^2 + Omega^3*gamma^3*r^4 - 4*Omega^2*a*gamma^3*m*r)*rho(r))*sin(th)^2 + (2*Omega*gamma^3*m*r - Omega*gamma^3*r^2)*rho(r))/(a^2*cos(th)^2 + r^2) ∂/∂ph

Now as assumed if $\Omega \approx 0$, then the $\frac{\partial}{\partial \phi}$ component of $T^{ab}u_a$ yields zero as shown below. Note that we could have set $\Omega = 0$ in $u^a$ to begin with and saved some computation

In [27]:
temp = mass_energy_currrent[3].expr()
temp = temp.subs(Omega = 0); temp

0

On the other hand, the $\frac{\partial}{\partial t}$ component yields

In [28]:
temp = mass_energy_currrent[0].expr()  # The [0] only shows the time coefficient of the field.
temp.subs(Omega = 0).factor()

-(a^2*cos(th)^2 - 2*m*r + r^2)*gamma^3*rho(r)/(a^2*cos(th)^2 + r^2)

The above implies that for $\Omega \approx 0$, $T^{a b} u_{a}$ is dominated by $-\frac{\left(a^{2} \cos ^{2} \theta-2 M r+r^{2}\right) \gamma^{2}}{a^{2} \cos ^{2} \theta+r^{2}} \rho u^{t}$ 

### Calculating unit hypersurface normal 1-form 

$n_b = \frac{-dt}{\sqrt{-g^{-1}(dt, dt)}}$

 The negative sign at the front is to keep n future directed

Next, first calculate $g^{-1}$

In [29]:
g_inv = g.inverse(); g_inv.display()

inv_g = -(2*a^2*m*r*sin(th)^2 + a^2*r^2 + r^4 + (a^4 + a^2*r^2)*cos(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2) ∂/∂t⊗∂/∂t - 2*a*m*r/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2) ∂/∂t⊗∂/∂ph + (a^2 - 2*m*r + r^2)/(a^2*cos(th)^2 + r^2) ∂/∂r⊗∂/∂r + 1/(a^2*cos(th)^2 + r^2) ∂/∂th⊗∂/∂th - 2*a*m*r/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2) ∂/∂ph⊗∂/∂t + (a^2*cos(th)^2 - 2*m*r + r^2)/(2*a^2*m*r*sin(th)^4 - (2*a^2*m*r - a^2*r^2 + 2*m*r^3 - r^4 - (a^4 + a^2*r^2)*cos(th)^2)*sin(th)^2) ∂/∂ph⊗∂/∂ph

The dual basis associated with $\xi = \frac{\partial}{\partial t}$ is 

In [30]:
n = BL.coframe()[0] ; n

1-form dt on the 4-dimensional Lorentzian manifold M

Normalise it

In [31]:
n_unit =  - n / sqrt(-g_inv(n, n))
n_unit.display()

-sqrt(a^2*cos(th)^2 + r^2)*sqrt(abs(a^2 - 2*m*r + r^2))/sqrt(2*a^2*m*r*sin(th)^2 + a^2*r^2 + r^4 + (a^4 + a^2*r^2)*cos(th)^2) dt

Check that it is normalised

In [32]:
g_inv(n_unit, n_unit).display()

M → ℝ
(t, r, th, ph) ↦ -abs(a^2 - 2*m*r + r^2)/(a^2 - 2*m*r + r^2)

Declare the *Riemannian* Submanifold associated with the spacelike hypersurface $\Sigma_t$

In [33]:
N = Manifold(3, 'N', structure='Riemannian'); print(N)

3-dimensional Riemannian manifold N


The induced BL chart is 

In [34]:
BL_N.<r_n,th_n,ph_N> = N.chart(r'r_N th_N:(0,pi):\theta_N ph_N:(0,2*pi):\phi_N')
BL_N

Chart (N, (r_N, th_N, ph_N))

In [35]:
BL_N.coord_range()

r_N: (-oo, +oo); th_N: (0, pi); ph_N: (0, 2*pi)

Declaring the induced metric on $N$

In [36]:
h = N.metric(name = 'h'); print(h)

Riemannian metric h on the 3-dimensional Riemannian manifold N


Define metric below (calculated by setting $dt = 0$ in $g_{ab}$

In [37]:
rho2 = r^2 + (a*cos(th))^2
Delta = r^2 - 2*m*r + a^2

h[0,0] = rho2/Delta
h[1,1] = rho2
h[2,2] = (r^2 + a^2 + 2*m*r*(a*sin(th))^2/rho2)*sin(th)^2

# # ---- For induced metric from Minkowski --------
# h[0,0] = 1
# h[1,1] = 1
# h[2,2] = 1

In [38]:
h.display()

h = (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2) dr_N⊗dr_N + (a^2*cos(th)^2 + r^2) dth_N⊗dth_N + (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2 dph_N⊗dph_N

Calculating $\sqrt{{\rm det}(h)}$ which is called $\sqrt{g_{\Sigma_t}} = \sqrt{{\rm det}(g_{\mu \nu})}$ in the paper. Note that the induced metric $g_{\mu \nu}$ is called $h$ here.

In [39]:
sqrt(det(h)).display()

N → ℝ
(r_N, th_N, ph_N) ↦ sqrt(2*a^2*m*r*sin(th)^2 + a^2*r^2 + r^4 + (a^4 + a^2*r^2)*cos(th)^2)*sqrt(a^2*cos(th)^2 + r^2)*sin(th)/sqrt(abs(a^2 - 2*m*r + r^2))

Extract the expression

In [40]:
sqrt_det_h = sqrt(det(h)).expr(); sqrt_det_h

sqrt(2*a^2*m*r*sin(th)^2 + a^2*r^2 + r^4 + (a^4 + a^2*r^2)*cos(th)^2)*sqrt(a^2*cos(th)^2 + r^2)*sin(th)/sqrt(abs(a^2 - 2*m*r + r^2))

 ----------------------------  Calculating $T^{ab} u_a n_b$  ----------------------------

In [41]:
mass_energy_currrent.contract(n_unit).display()

M → ℝ
(t, r, th, ph) ↦ -(2*Omega^2*a^2*gamma^3*m*r*rho(r)*sin(th)^4 - a^2*gamma^3*cos(th)^2*rho(r) + ((Omega^2*a^4*gamma^3 + Omega^2*a^2*gamma^3*r^2)*cos(th)^2*rho(r) + (Omega^2*a^2*gamma^3*r^2 + Omega^2*gamma^3*r^4 - 4*Omega*a*gamma^3*m*r)*rho(r))*sin(th)^2 + (2*gamma^3*m*r - gamma^3*r^2)*rho(r))*sqrt(abs(a^2 - 2*m*r + r^2))/(sqrt(2*a^2*m*r*sin(th)^2 + a^2*r^2 + r^4 + (a^4 + a^2*r^2)*cos(th)^2)*sqrt(a^2*cos(th)^2 + r^2))

Now calculating the full expression $\sqrt{{\rm det}(h)} T^{ab} u_a n_b$. We will call it $dM$ although in the paper $dM$ also has to integrated over $\theta$ and $\phi$ coordinates

In [42]:
dM = sqrt(det(h)).expr() * mass_energy_currrent.contract(n_unit).expr()

Substitute $\Omega = 0$

In [43]:
dM = dM.subs(Omega = 0); dM

(a^2*gamma^3*cos(th)^2*rho(r) - (2*gamma^3*m*r - gamma^3*r^2)*rho(r))*sin(th)

Integrating over the $\theta$ and $\phi$ coordiante (this is not needed but just performing a check)

In [44]:
integrate(integrate( dM, (th, 0, pi)) , (ph, 0 , 2*pi))

4/3*pi*(a^2*gamma^3*rho(r) - 3*(2*gamma^3*m*r - gamma^3*r^2)*rho(r))

### Next finding $dJ$ 

The second Killing vector field associated with $g_{ab}$ is $\eta^b$ and has the components (0,0,0,1) as defined below

In [45]:
eta = BL.frame()[3]; eta

Vector field ∂/∂ph on the 4-dimensional Lorentzian manifold M

The covector/1-form $\eta_b$ associated with $\eta^b$ is 

In [46]:
eta.down(g).display()

-2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dt + (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2 dph

Calculating $T^{ab} \eta_b$

In [47]:
angular_momentum_current = T.contract( - eta.down(g))  # Negative sign again
angular_momentum_current.display()
# angular_momentum_current[0].factor()

-(2*Omega*a^2*gamma^2*m*r*rho(r)*sin(th)^4 + ((Omega*a^4*gamma^2 + Omega*a^2*gamma^2*r^2)*cos(th)^2*rho(r) + (Omega*a^2*gamma^2*r^2 + Omega*gamma^2*r^4 - 2*a*gamma^2*m*r)*rho(r))*sin(th)^2)/(a^2*cos(th)^2 + r^2) ∂/∂t - (2*Omega^2*a^2*gamma^2*m*r*rho(r)*sin(th)^4 + ((Omega^2*a^4*gamma^2 + Omega^2*a^2*gamma^2*r^2)*cos(th)^2*rho(r) + (Omega^2*a^2*gamma^2*r^2 + Omega^2*gamma^2*r^4 - 2*Omega*a*gamma^2*m*r)*rho(r))*sin(th)^2)/(a^2*cos(th)^2 + r^2) ∂/∂ph

----------------------------  Calculating $T^{ab} \eta_a n_b$  ----------------------------

In [48]:
angular_momentum_current.contract(n_unit).display()

M → ℝ
(t, r, th, ph) ↦ (2*Omega*a^2*gamma^2*m*r*rho(r)*sin(th)^4 + ((Omega*a^4*gamma^2 + Omega*a^2*gamma^2*r^2)*cos(th)^2*rho(r) + (Omega*a^2*gamma^2*r^2 + Omega*gamma^2*r^4 - 2*a*gamma^2*m*r)*rho(r))*sin(th)^2)*sqrt(abs(a^2 - 2*m*r + r^2))/(sqrt(2*a^2*m*r*sin(th)^2 + a^2*r^2 + r^4 + (a^4 + a^2*r^2)*cos(th)^2)*sqrt(a^2*cos(th)^2 + r^2))

Calculating $dJ$.  We are calling the below quantity $dJ$ although in the paper $dJ$ has been integrated over $\theta$ and $\phi$ coordinates

In [49]:
dJ = sqrt(det(h)).expr() * angular_momentum_current.contract(n_unit).expr()
dJ.factor()

(Omega*a^4*cos(th)^2 + Omega*a^2*r^2*cos(th)^2 + 2*Omega*a^2*m*r*sin(th)^2 + Omega*a^2*r^2 + Omega*r^4 - 2*a*m*r)*gamma^2*rho(r)*sin(th)^3

In the non-relativistic case this should yield

In [50]:
dJ.subs(a = 0, gamma = 1).factor()

Omega*r^4*rho(r)*sin(th)^3

Doing the integration ove the $\theta$ and $\phi$ coordiantes

In [51]:
temp = dJ.factor()
integrate(integrate( temp, (th, 0, pi)) , (ph, 0 , 2*pi)) 

8/15*pi*(Omega*a^4 + 6*Omega*a^2*r^2 + 5*Omega*r^4 + 2*(4*Omega*a^2 - 5*a)*m*r)*gamma^2*rho(r)

### Calculating $\gamma$, see Eq. 8 in paper

Calculating the norm square of $u$

In [52]:
u_norm_sq = g(u,u); u_norm_sq.display()

g(u,u): M → ℝ
   (t, r, th, ph) ↦ (2*Omega^2*a^2*gamma^2*m*r*sin(th)^4 - a^2*gamma^2*cos(th)^2 + 2*gamma^2*m*r - gamma^2*r^2 + (Omega^2*a^2*gamma^2*r^2 + Omega^2*gamma^2*r^4 - 4*Omega*a*gamma^2*m*r + (Omega^2*a^4*gamma^2 + Omega^2*a^2*gamma^2*r^2)*cos(th)^2)*sin(th)^2)/(a^2*cos(th)^2 + r^2)

In [53]:
# The below returns the expression stored in u_norm_sq
u_norm_sq = u_norm_sq.expr()

In [54]:
u_norm_sq = u_norm_sq.subs(Omega = 0) #; u_norm_sq

Find $\gamma$ below. It gives two roots

In [55]:
solve(u_norm_sq == -1, gamma)#[1].factor()

[gamma == -sqrt(a^2*cos(th)^2/(a^2*cos(th)^2 - 2*m*r + r^2) + r^2/(a^2*cos(th)^2 - 2*m*r + r^2)), gamma == sqrt(a^2*cos(th)^2/(a^2*cos(th)^2 - 2*m*r + r^2) + r^2/(a^2*cos(th)^2 - 2*m*r + r^2))]

with the positive one being

In [56]:
solve(u_norm_sq == -1, gamma)[1].factor()

gamma == sqrt((a^2*cos(th)^2 + r^2)/(a^2*cos(th)^2 - 2*m*r + r^2))